<a href="https://colab.research.google.com/github/softcomput/cp-colab/blob/main/cpp_from_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Getting started**

1. Install Google OR-Tools.
2. Create small data model or large data model.
3. Run carpooling problem from work.

In [3]:
!pip install ortools

In [12]:
import logging
import re

from ortools.linear_solver import pywraplp
import numpy as np


class CPPFromWork:
    """CPPFromWork

    Attributes:
        data (dict):
            Vs (list): servers
            Vc (list): clients
            Q (list): capacities
            T (list): maximum ride times
            distance (2-d list): distance for each server and client
            duration (2-d list): duration for each server and client
            risk (2-d list): risk for each server and client
    """

    def __init__(self, data):
        self.d = self.__format_data(data)

    def __format_data(self, data):
        """ __format_data

        Format data model.
        """
        distance = []
        for vs_d in data['distance'][0:len(data['Vs'])]:
            distance.append(vs_d)
            distance.extend(data['distance'][len(data['Vs']):])
        data['distance'] = distance

        duration = []
        for vs_d in data['duration'][0:len(data['Vs'])]:
            duration.append(vs_d)
            duration.extend(data['duration'][len(data['Vs']):])
        data['duration'] = duration

        distance_vc = []
        for vc in range(len(data['Vc'])):
            distance_vc.append(data['distance'][vc + 1][0])
        data['distance_vc'] = distance_vc

        return data

    def __create_variables(self, solver):
        """ __create_variables

        Create variables.

        Returns:
            x (list): x^k_ij, k in Vs
            y (list): y_i, i in Vc
            u (list): u^k_i, 0 <= u^k_i <= Qk, i in V
        """
        x = []
        u = []
        for idx, k in enumerate(self.d['Vs']):
            ai = [0]
            ai.extend(self.d['Vc'])
            aj = [k]
            aj.extend(self.d['Vc'])
            for j in aj:
                x.append([
                    solver.IntVar(0, 1, 'x{}({},{})'.format(k, i, j))
                    for i in ai])
            u.append([
                solver.IntVar(1, self.d['Q'][idx], 'u{}({})'.format(k, j))
                for j in aj])
        y = [solver.IntVar(0, 1, 'y({})'.format(i)) for i in self.d['Vc']]
        return x, y, u

    def __create_constraints(self, solver, x, y, u):
        """ __create_constraints

        Create constraints.
        """
        n = len(self.d['Vc']) + 1
        for vs in range(len(self.d['Vs'])):
            k = vs * n
            # Each employee leaves the workspace.
            solver.Add(solver.Sum([x[j][0] for j in range(k, k + n)]) == 1)
            # Each employee arrives at home.
            solver.Add(solver.Sum(x[k]) == 1)
            # Continuity.
            for i in range(1, len(self.d['Vc']) + 1):
                solver.Add(
                    solver.Sum(x[k + i]) -
                    solver.Sum([x[j][i] for j in range(k, k + n)]) == 0)
            # Capacity.
            solver.Add(
                solver.Sum([x_k for i in range(n) for x_k in x[k + i]]) <=
                self.d['Q'][vs])
            # Maximum ride time.
            solver.Add(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    self.d['duration'][k:k + n], x[k:k + n])]) <=
                self.d['T'][vs])
            # Own arc is not passed.
            for i in range(1, len(self.d['Vc']) + 1):
                solver.Add(x[k + i][i] == 0)
            # Subtour elimination constraints by MTZ.
            for i in range(1, len(u[vs])):
                solver.Add(
                    u[vs][i] + 1 - self.d['Q'][vs] * (1 - x[k][i]) <= u[vs][0])
            for i in range(1, len(u[vs])):
                for j in range(1, len(u[vs])):
                    if str(u[vs][i]) != str(u[vs][j]):
                        solver.Add(
                            u[vs][j] + 1 -
                            self.d['Q'][vs] * (1 - x[k + i][j]) <=
                            u[vs][i])
            solver.Add(u[vs][0] == self.d['Q'][vs])

        # Each client is either picked up by a server or is left unserviced.
        for vc in range(len(self.d['Vc'])):
            solver.Add(
                solver.Sum([x_k for vs in range(len(self.d['Vs']))
                           for x_k in x[vs * n + vc + 1]]) + y[vc] == 1)

    def __create_objective(self, solver, x, y):
        """ __create_objective

        Create objective.
        """
        if self.d.get('risk'):
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    np.array(self.d['risk']) *
                    np.array(self.d['distance']), x)]) +
                solver.Sum([np.inner(np.array(self.d['distance_vc']) * 2, y)]))
        else:
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    self.d['distance'], x)]) +
                solver.Sum([np.inner(np.array(self.d['distance_vc']) * 2, y)]))

    def solve(self):
        """ solve

        Solve CPP.
        """
        solver = pywraplp.Solver(
            'cp', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

        x, y, u = self.__create_variables(solver)
        self.__create_constraints(solver, x, y, u)
        self.__create_objective(solver, x, y)

        status = solver.Solve()

        paths = {}
        if status == pywraplp.Solver.OPTIMAL:
            n = len(self.d['Vc']) + 1
            root = []
            for i in range(len(self.d['Vs']) * n):
                for j in range(n):
                    if x[i][j].solution_value():
                        root.append(x[i][j].name())
                        r = re.findall(r'\w+', x[i][j].name())
                        r[0] = r[0].lstrip('x')
                        if not r[0] in paths:
                            paths[r[0]] = []
                        paths[r[0]].append((r[1], r[2]))
            paths = self.__format_paths(paths)
            clients_not_pickedup = []
            for i in range(len(self.d['Vc'])):
                if y[i].solution_value():
                    r = re.findall(r'\w+', y[i].name())
                    clients_not_pickedup.append(r[1])
            arrival_times = []
            for i in range(len(self.d['Vs'])):
                arrival_times.append([
                    (u[i][j].name(), u[i][j].solution_value())
                    for j in range(len(u[i]))])
            logging.info(
                'Root %s, clients not pickedup %s,\
                and arrival_times %s solved in %d variables, \
                %d constraints, %f objective value, %f milliseconds, \
                %d iterations, and %d branch-and-bound nodes.' % (
                    root, clients_not_pickedup, arrival_times,
                    solver.NumVariables(),
                    solver.NumConstraints(),
                    solver.Objective().Value(),
                    solver.wall_time(), solver.iterations(), solver.nodes()))
        else:
            logging.info('The problem does not have an optimal solution.')
        m, tdr, dti = self.__evaluate(x, y)
        er = self.__evaluate_accident_evasion_rate(x, y)
        logging.info('evasion_rate: %s', er)
        print('evasion_rate: %s', er)
        return {
            'feasible_paths': paths,
            '%m': m,
            '%tdr': tdr,
            '%dti': dti,
            'clients_not_pickedup': clients_not_pickedup
        }

    def __format_paths(self, paths):
        for i in paths:
            clients = []
            prev = paths[i][0][0]
            for _ in paths[i]:
                if not prev == '0':
                    for j in paths[i]:
                        if prev == j[1]:
                            clients.insert(0, j[1])
                            prev = j[0]
                            break
            paths[i] = clients
        return paths

    def __evaluate(self, x, y):
        total_distance = 0
        total_duration = 0
        original_distance = 0
        original_duration_vs = 0
        n = len(self.d['Vc']) + 1
        for i in range(len(self.d['Vs']) * n):
            for j in range(n):
                if x[i][j].solution_value():
                    total_distance += self.d['distance'][i][j]
                    total_duration += self.d['duration'][i][j]
        for i in range(len(self.d['Vs'])):
            original_distance += self.d['distance'][i * n][0]
            original_duration_vs += self.d['duration'][i * n][0]
        not_pickup_clients = 0
        for i in range(len(self.d['Vc'])):
            if y[i].solution_value() == 1:
                not_pickup_clients += 1
                total_distance += self.d['distance'][i + 1][0]
            original_distance += self.d['distance'][i + 1][0]

        m = (len(self.d['Vc']) - not_pickup_clients) / len(self.d['Vc'])
        tdr = 1 - total_distance / original_distance
        dti = total_duration / original_duration_vs
        return m, tdr, dti

    def __evaluate_accident_evasion_rate(self, x, y):
        if not self.d.get('accidentCount'):
            return 0
        total_accident_count = 0
        original_accident_count = 0
        n = len(self.d['Vc']) + 1
        for i in range(len(self.d['Vs']) * n):
            for j in range(n):
                if x[i][j].solution_value():
                    total_accident_count += self.d['accidentCount'][i][j]
        for i in range(len(self.d['Vs'])):
            original_accident_count += self.d['accidentCount'][i * n][0]
        for i in range(len(self.d['Vc'])):
            if y[i].solution_value() == 1:
                total_accident_count += self.d['accidentCount'][i + 1][0]
            original_accident_count += self.d['accidentCount'][i + 1][0]

        evasion_rate = 1 - total_accident_count / original_accident_count
        return evasion_rate


data = create_data_model()
output = CPPFromWork(data).solve()
print(output)

evasion_rate: %s 0.6486486486486487
{'feasible_paths': {'1': ['12'], '2': ['8', '15', '14', '7'], '3': ['10', '9', '11', '13'], '4': [], '5': ['18', '17', '20'], '6': ['19', '16']}, '%m': 1.0, '%tdr': 0.5769843851659076, '%dti': 1.609585853885528, 'clients_not_pickedup': []}


In [11]:
def create_data_model():
    data = {}
    data['Vs'] = [1,2,3,4,5,6]
    data['Vc'] = [7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    data['Q'] = [5,5,5,5,5,5]
    data['T'] = [3600,3600,3600,3600,3600,3600]
    data['Ls'] = [7200,7200,7200,7200,7200,7200]
    data['Lc'] = [7200,7200,7200,7200,7200,7200,7200,7200,7200,7200,7200,7200,7200,7200]
    data['Ss'] = [0,0,0,0,0,0]
    data['Sc'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    data['distance'] = [[6556,6850,3692,6225,8220,5232,7395,8073,5532,3692,21680,20400,10633,13076,20859],[8812,2675,4551,2316,3768,3236,9817,4137,2960,4551,24102,22822,13055,15498,23281],[9932,5296,5995,3077,4573,4559,10938,2716,5581,5995,29160,27880,14176,16619,28339],[4751,3190,1853,2841,4658,1415,5756,3758,3475,1853,20041,18761,8994,11437,19220],[16792,16773,16974,14233,16061,15154,11671,14223,17058,16974,5116,4181,10290,7804,2683],[15975,22778,21441,22300,26719,21003,10750,23216,23063,21441,785,2296,7860,5375,3181],[7734,0,3652,2239,4351,3159,8837,4720,809,3652,23122,21842,12075,14518,22301],[6006,3652,0,2717,4712,1724,6844,4663,3627,0,21129,19849,10082,12525,20308],[7230,2239,2717,0,2102,1576,8235,2159,2510,3013,22520,21240,11473,13916,21699],[12455,4351,4712,2102,0,5231,21587,4119,6164,6667,34435,33155,24824,27464,33614],[5959,3159,1724,1576,5231,0,7062,3153,3656,2034,21347,20067,10300,12743,20526],[2068,8837,6844,8235,21587,7062,0,9309,9156,7534,9521,14135,3238,5681,14594],[9005,4720,4663,2159,4119,3153,9309,0,4515,4968,23580,22300,12533,14976,17207],[8019,809,3627,2510,6164,3656,9156,4515,0,2662,23407,22127,12360,14803,22586],[6006,3652,0,3013,6667,2034,7534,4968,2662,0,21129,19849,10082,12525,20308],[15224,23122,21129,22520,34435,21347,9521,23580,23407,21129,0,1545,6625,4139,2430],[13928,21842,19849,21240,33155,20067,14135,22300,22127,19849,1545,0,7138,4652,1783],[5371,12075,10082,11473,24824,10300,3238,12533,12360,10082,6625,7138,0,2511,7262],[7754,14518,12525,13916,27464,12743,5681,14976,14803,12525,4139,4652,2511,0,5045],[14863,22301,20308,21699,33614,20526,14594,17207,22586,20308,2430,1783,7262,5045,0]]
    data['duration'] = [[722,915,461,895,1041,697,812,1115,723,461,1787,1629,1211,1491,1702],[1087,388,676,405,434,487,1210,745,392,676,2185,2027,1609,1889,2100],[1293,872,1020,594,619,783,1415,446,876,1020,2310,2152,1814,2094,2225],[504,580,370,543,587,293,626,648,584,370,1601,1443,1025,1305,1516],[1442,2086,2082,1776,1649,1823,1366,1773,2090,2082,842,745,1518,1178,466],[1399,2272,2062,2190,2170,1985,1342,2295,2276,2062,141,496,1109,770,594],[1014,0,579,433,514,524,1138,825,167,579,2113,1955,1537,1817,2028],[671,579,0,496,643,298,758,857,521,0,1733,1575,1157,1437,1648],[880,433,496,0,233,301,1002,474,414,538,1977,1819,1401,1681,1892],[969,514,643,233,0,567,1119,664,681,804,1952,1794,1518,1619,1867],[687,524,298,301,567,0,811,617,535,317,1786,1628,1210,1490,1701],[263,1138,758,1002,1119,811,0,1159,1140,926,1200,969,399,679,1042],[992,825,857,474,664,617,1159,0,832,839,2167,2009,1591,1871,2014],[1019,167,521,414,681,535,1140,832,0,396,2118,1960,1542,1822,2033],[671,579,0,538,804,317,926,839,396,0,1733,1575,1157,1437,1648],[1298,2113,1733,1977,1952,1786,1200,2167,2118,1733,0,395,1036,697,493],[1138,1955,1575,1819,1794,1628,969,2009,1960,1575,395,0,1165,826,470],[723,1537,1157,1401,1518,1210,399,1591,1542,1157,1036,1165,0,347,1029],[948,1817,1437,1681,1619,1490,679,1871,1822,1437,697,826,347,0,784],[1182,2028,1648,1892,1867,1701,1042,2014,2033,1648,493,470,1029,784,0]]
    data['risk'] = [[2,1,1,1.2777777777777777,1,1,1,1,1,1,1,1,1.3888888888888888,1.2222222222222223,1.2777777777777777],[2,0,1,1.2777777777777777,1,1.2222222222222223,1,1.2222222222222223,1,1,1,1,1.3888888888888888,1.2222222222222223,1],[2,1,0,1.3888888888888888,1,1,1,1,1,1,1,1,1.3888888888888888,1.2222222222222223,1],[2,1.2777777777777777,1.3888888888888888,0,1,1.3888888888888888,1.2222222222222223,1,1.2222222222222223,1.2777777777777777,1.2222222222222223,1,1.3888888888888888,1.2222222222222223,1.2777777777777777],[1,1,1,1,0,1.3888888888888888,1,1,1.2222222222222223,1.2777777777777777,1,1,1,1,1],[2,1.2222222222222223,1,1.3888888888888888,1.3888888888888888,0,1,1,1.4444444444444444,1,1.2222222222222223,2,1.3888888888888888,1.2222222222222223,1],[1,1,1,1.2222222222222223,1,1,0,1.3333333333333333,1,1,1,1.4444444444444444,1,1.2222222222222223,1],[1.3333333333333333,1.2222222222222223,1,1,1,1,1.3333333333333333,0,1.2222222222222223,1.2222222222222223,1.2222222222222223,1,1.3888888888888888,1.2222222222222223,1.4444444444444444],[2,1,1,1.2222222222222223,1.2222222222222223,1.4444444444444444,1,1.2222222222222223,0,1,1,1,1.3888888888888888,1.2222222222222223,1],[2,1,1,1.2777777777777777,1.2777777777777777,1,1,1.2222222222222223,1,0,1,1,1.3888888888888888,1.2222222222222223,1],[1.2222222222222223,1,1,1.2222222222222223,1,1.2222222222222223,1,1.2222222222222223,1,1,0,1.6111111111111112,1,1.2222222222222223,1.2777777777777777],[2,1,1,1,1,2,1.4444444444444444,1,1,1,1.6111111111111112,0,1,1.2222222222222223,1.2777777777777777],[1.3888888888888888,1.3888888888888888,1.3888888888888888,1.3888888888888888,1,1.3888888888888888,1,1.3888888888888888,1.3888888888888888,1.3888888888888888,1,1,0,1,1.2777777777777777],[1.3888888888888888,1.2222222222222223,1.2222222222222223,1.2222222222222223,1,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1,0,1.2777777777777777],[1,1,1,1.2777777777777777,1,1,1,1.4444444444444444,1,1,1.2777777777777777,1.2777777777777777,1.2777777777777777,1.2777777777777777,0],[2.25,1.25,1.25,1.6944444444444444,1.25,1.4722222222222223,1.25,1.5833333333333333,1.25,1.25,1.8611111111111112,1.8611111111111112,1.25,1.4722222222222223,1.25],[2.25,0,1.25,1.5277777777777777,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,1.25,0,1.6388888888888888,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,1.5277777777777777,1.6388888888888888,0,1.25,1.6388888888888888,1.4722222222222223,1.25,1.4722222222222223,1.5277777777777777,1.4722222222222223,1.25,1.6388888888888888,1.4722222222222223,1.5277777777777777],[1.25,1.25,1.25,1.25,0,1.6388888888888888,1.25,1.25,1.4722222222222223,1.5277777777777777,1.25,1.25,1.25,1.25,1.25],[2.25,1.4722222222222223,1.25,1.6388888888888888,1.6388888888888888,0,1.25,1.25,1.6944444444444444,1.25,1.4722222222222223,2.25,1.6388888888888888,1.4722222222222223,1.25],[1.25,1.25,1.25,1.4722222222222223,1.25,1.25,0,1.5833333333333333,1.25,1.25,1.25,1.6944444444444444,1.25,1.4722222222222223,1.25],[1.5833333333333333,1.4722222222222223,1.25,1.25,1.25,1.25,1.5833333333333333,0,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.25,1.6388888888888888,1.4722222222222223,1.6944444444444444],[2.25,1.25,1.25,1.4722222222222223,1.4722222222222223,1.6944444444444444,1.25,1.4722222222222223,0,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,1.25,1.25,1.5277777777777777,1.5277777777777777,1.25,1.25,1.4722222222222223,1.25,0,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[1.4722222222222223,1.25,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.25,0,1.8611111111111112,1.25,1.4722222222222223,1.5277777777777777],[2.25,1.25,1.25,1.25,1.25,2.25,1.6944444444444444,1.25,1.25,1.25,1.8611111111111112,0,1.25,1.4722222222222223,1.5277777777777777],[1.6388888888888888,1.6388888888888888,1.6388888888888888,1.6388888888888888,1.25,1.6388888888888888,1.25,1.6388888888888888,1.6388888888888888,1.6388888888888888,1.25,1.25,0,1.25,1.5277777777777777],[1.6388888888888888,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.25,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.25,0,1.5277777777777777],[1.25,1.25,1.25,1.5277777777777777,1.25,1.25,1.25,1.6944444444444444,1.25,1.25,1.5277777777777777,1.5277777777777777,1.5277777777777777,1.5277777777777777,0],[2,1.2777777777777777,1.2222222222222223,1.2777777777777777,1.2777777777777777,1.2777777777777777,1.3333333333333333,1.3888888888888888,1.2777777777777777,1.2222222222222223,1,1,1.3888888888888888,1.2777777777777777,1],[2,0,1,1.2777777777777777,1,1.2222222222222223,1,1.2222222222222223,1,1,1,1,1.3888888888888888,1.2222222222222223,1],[2,1,0,1.3888888888888888,1,1,1,1,1,1,1,1,1.3888888888888888,1.2222222222222223,1],[2,1.2777777777777777,1.3888888888888888,0,1,1.3888888888888888,1.2222222222222223,1,1.2222222222222223,1.2777777777777777,1.2222222222222223,1,1.3888888888888888,1.2222222222222223,1.2777777777777777],[1,1,1,1,0,1.3888888888888888,1,1,1.2222222222222223,1.2777777777777777,1,1,1,1,1],[2,1.2222222222222223,1,1.3888888888888888,1.3888888888888888,0,1,1,1.4444444444444444,1,1.2222222222222223,2,1.3888888888888888,1.2222222222222223,1],[1,1,1,1.2222222222222223,1,1,0,1.3333333333333333,1,1,1,1.4444444444444444,1,1.2222222222222223,1],[1.3333333333333333,1.2222222222222223,1,1,1,1,1.3333333333333333,0,1.2222222222222223,1.2222222222222223,1.2222222222222223,1,1.3888888888888888,1.2222222222222223,1.4444444444444444],[2,1,1,1.2222222222222223,1.2222222222222223,1.4444444444444444,1,1.2222222222222223,0,1,1,1,1.3888888888888888,1.2222222222222223,1],[2,1,1,1.2777777777777777,1.2777777777777777,1,1,1.2222222222222223,1,0,1,1,1.3888888888888888,1.2222222222222223,1],[1.2222222222222223,1,1,1.2222222222222223,1,1.2222222222222223,1,1.2222222222222223,1,1,0,1.6111111111111112,1,1.2222222222222223,1.2777777777777777],[2,1,1,1,1,2,1.4444444444444444,1,1,1,1.6111111111111112,0,1,1.2222222222222223,1.2777777777777777],[1.3888888888888888,1.3888888888888888,1.3888888888888888,1.3888888888888888,1,1.3888888888888888,1,1.3888888888888888,1.3888888888888888,1.3888888888888888,1,1,0,1,1.2777777777777777],[1.3888888888888888,1.2222222222222223,1.2222222222222223,1.2222222222222223,1,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1,0,1.2777777777777777],[1,1,1,1.2777777777777777,1,1,1,1.4444444444444444,1,1,1.2777777777777777,1.2777777777777777,1.2777777777777777,1.2777777777777777,0],[2.25,1.6944444444444444,1.25,1.4722222222222223,1.25,1.25,1.25,1.25,1.6944444444444444,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,0,1.25,1.5277777777777777,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,1.25,0,1.6388888888888888,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,1.5277777777777777,1.6388888888888888,0,1.25,1.6388888888888888,1.4722222222222223,1.25,1.4722222222222223,1.5277777777777777,1.4722222222222223,1.25,1.6388888888888888,1.4722222222222223,1.5277777777777777],[1.25,1.25,1.25,1.25,0,1.6388888888888888,1.25,1.25,1.4722222222222223,1.5277777777777777,1.25,1.25,1.25,1.25,1.25],[2.25,1.4722222222222223,1.25,1.6388888888888888,1.6388888888888888,0,1.25,1.25,1.6944444444444444,1.25,1.4722222222222223,2.25,1.6388888888888888,1.4722222222222223,1.25],[1.25,1.25,1.25,1.4722222222222223,1.25,1.25,0,1.5833333333333333,1.25,1.25,1.25,1.6944444444444444,1.25,1.4722222222222223,1.25],[1.5833333333333333,1.4722222222222223,1.25,1.25,1.25,1.25,1.5833333333333333,0,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.25,1.6388888888888888,1.4722222222222223,1.6944444444444444],[2.25,1.25,1.25,1.4722222222222223,1.4722222222222223,1.6944444444444444,1.25,1.4722222222222223,0,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,1.25,1.25,1.5277777777777777,1.5277777777777777,1.25,1.25,1.4722222222222223,1.25,0,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[1.4722222222222223,1.25,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.25,0,1.8611111111111112,1.25,1.4722222222222223,1.5277777777777777],[2.25,1.25,1.25,1.25,1.25,2.25,1.6944444444444444,1.25,1.25,1.25,1.8611111111111112,0,1.25,1.4722222222222223,1.5277777777777777],[1.6388888888888888,1.6388888888888888,1.6388888888888888,1.6388888888888888,1.25,1.6388888888888888,1.25,1.6388888888888888,1.6388888888888888,1.6388888888888888,1.25,1.25,0,1.25,1.5277777777777777],[1.6388888888888888,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.25,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.25,0,1.5277777777777777],[1.25,1.25,1.25,1.5277777777777777,1.25,1.25,1.25,1.6944444444444444,1.25,1.25,1.5277777777777777,1.5277777777777777,1.5277777777777777,1.5277777777777777,0],[1,1,1,1.3888888888888888,1,1.3888888888888888,1,1.3888888888888888,1,1,1.2777777777777777,1.6111111111111112,1,1.2222222222222223,1.2777777777777777],[2,0,1,1.2777777777777777,1,1.2222222222222223,1,1.2222222222222223,1,1,1,1,1.3888888888888888,1.2222222222222223,1],[2,1,0,1.3888888888888888,1,1,1,1,1,1,1,1,1.3888888888888888,1.2222222222222223,1],[2,1.2777777777777777,1.3888888888888888,0,1,1.3888888888888888,1.2222222222222223,1,1.2222222222222223,1.2777777777777777,1.2222222222222223,1,1.3888888888888888,1.2222222222222223,1.2777777777777777],[1,1,1,1,0,1.3888888888888888,1,1,1.2222222222222223,1.2777777777777777,1,1,1,1,1],[2,1.2222222222222223,1,1.3888888888888888,1.3888888888888888,0,1,1,1.4444444444444444,1,1.2222222222222223,2,1.3888888888888888,1.2222222222222223,1],[1,1,1,1.2222222222222223,1,1,0,1.3333333333333333,1,1,1,1.4444444444444444,1,1.2222222222222223,1],[1.3333333333333333,1.2222222222222223,1,1,1,1,1.3333333333333333,0,1.2222222222222223,1.2222222222222223,1.2222222222222223,1,1.3888888888888888,1.2222222222222223,1.4444444444444444],[2,1,1,1.2222222222222223,1.2222222222222223,1.4444444444444444,1,1.2222222222222223,0,1,1,1,1.3888888888888888,1.2222222222222223,1],[2,1,1,1.2777777777777777,1.2777777777777777,1,1,1.2222222222222223,1,0,1,1,1.3888888888888888,1.2222222222222223,1],[1.2222222222222223,1,1,1.2222222222222223,1,1.2222222222222223,1,1.2222222222222223,1,1,0,1.6111111111111112,1,1.2222222222222223,1.2777777777777777],[2,1,1,1,1,2,1.4444444444444444,1,1,1,1.6111111111111112,0,1,1.2222222222222223,1.2777777777777777],[1.3888888888888888,1.3888888888888888,1.3888888888888888,1.3888888888888888,1,1.3888888888888888,1,1.3888888888888888,1.3888888888888888,1.3888888888888888,1,1,0,1,1.2777777777777777],[1.3888888888888888,1.2222222222222223,1.2222222222222223,1.2222222222222223,1,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1.2222222222222223,1,0,1.2777777777777777],[1,1,1,1.2777777777777777,1,1,1,1.4444444444444444,1,1,1.2777777777777777,1.2777777777777777,1.2777777777777777,1.2777777777777777,0],[1.25,1.25,1.4722222222222223,1.25,1.4722222222222223,1.4722222222222223,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.8611111111111112,1.25,1.4722222222222223,1.5277777777777777],[2.25,0,1.25,1.5277777777777777,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,1.25,0,1.6388888888888888,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,1.5277777777777777,1.6388888888888888,0,1.25,1.6388888888888888,1.4722222222222223,1.25,1.4722222222222223,1.5277777777777777,1.4722222222222223,1.25,1.6388888888888888,1.4722222222222223,1.5277777777777777],[1.25,1.25,1.25,1.25,0,1.6388888888888888,1.25,1.25,1.4722222222222223,1.5277777777777777,1.25,1.25,1.25,1.25,1.25],[2.25,1.4722222222222223,1.25,1.6388888888888888,1.6388888888888888,0,1.25,1.25,1.6944444444444444,1.25,1.4722222222222223,2.25,1.6388888888888888,1.4722222222222223,1.25],[1.25,1.25,1.25,1.4722222222222223,1.25,1.25,0,1.5833333333333333,1.25,1.25,1.25,1.6944444444444444,1.25,1.4722222222222223,1.25],[1.5833333333333333,1.4722222222222223,1.25,1.25,1.25,1.25,1.5833333333333333,0,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.25,1.6388888888888888,1.4722222222222223,1.6944444444444444],[2.25,1.25,1.25,1.4722222222222223,1.4722222222222223,1.6944444444444444,1.25,1.4722222222222223,0,1.25,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[2.25,1.25,1.25,1.5277777777777777,1.5277777777777777,1.25,1.25,1.4722222222222223,1.25,0,1.25,1.25,1.6388888888888888,1.4722222222222223,1.25],[1.4722222222222223,1.25,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.4722222222222223,1.25,1.25,0,1.8611111111111112,1.25,1.4722222222222223,1.5277777777777777],[2.25,1.25,1.25,1.25,1.25,2.25,1.6944444444444444,1.25,1.25,1.25,1.8611111111111112,0,1.25,1.4722222222222223,1.5277777777777777],[1.6388888888888888,1.6388888888888888,1.6388888888888888,1.6388888888888888,1.25,1.6388888888888888,1.25,1.6388888888888888,1.6388888888888888,1.6388888888888888,1.25,1.25,0,1.25,1.5277777777777777],[1.6388888888888888,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.25,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.4722222222222223,1.25,0,1.5277777777777777],[1.25,1.25,1.25,1.5277777777777777,1.25,1.25,1.25,1.6944444444444444,1.25,1.25,1.5277777777777777,1.5277777777777777,1.5277777777777777,1.5277777777777777,0]]
    data['accidentCount'] = [[18,0,0,5,0,0,0,0,0,0,0,0,7,4,5],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[18,0,0,8,0,4,0,6,0,0,11,11,0,4,0],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[18,5,4,5,5,5,6,7,5,4,0,0,7,5,0],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[18,8,0,4,0,0,0,0,8,0,0,0,7,4,0],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[0,0,0,7,0,7,0,7,0,0,5,11,0,4,5],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[0,0,4,0,4,4,0,4,0,4,0,11,0,4,5],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0]]
    return data

# def create_data_model():
#     """Stores the data for the problem."""
#     data = {}
#     data['Vs'] = ['1', '2']
#     data['Vc'] = ['3', '4', '5', '6', '7', '8', '9']
#     data['Q'] = [7, 5]
#     data['T'] = [12, 12]
#     data['Ls'] = [12, 12]
#     data['Lc'] = [12, 12, 12, 12, 12, 12, 100]
#     data['Ss'] = [0, 0]
#     data['Sc'] = [0, 0, 0, 0, 0, 0, 0]
#     data['distance'] = [
#                         [10, 5, 7, 10, 1, 3, 10, 100],
#                         [10, 7, 9, 4, 11, 11, 12, 100],
#                         [5, 0, 2, 5, 4, 2, 5, 100],
#                         [3, 2, 0, 5, 6, 4, 3, 100],
#                         [6, 5, 5, 0, 9, 7, 8, 100],
#                         [9, 4, 6, 9, 0, 2, 9, 100],
#                         [7, 2, 4, 7, 2, 0, 7, 100],
#                         [1, 5, 3, 8, 9, 7, 0, 100],
#                         [100, 100, 100, 100, 100, 100, 100, 0],
#                         ]
#     data['duration'] = [
#                         [10, 5, 7, 10, 1, 3, 10, 100],
#                         [10, 7, 9, 4, 11, 11, 12, 100],
#                         [5, 0, 2, 5, 4, 2, 5, 100],
#                         [3, 2, 0, 5, 6, 4, 3, 100],
#                         [6, 5, 5, 0, 9, 7, 8, 100],
#                         [9, 4, 6, 9, 0, 2, 9, 100],
#                         [7, 2, 4, 7, 2, 0, 7, 100],
#                         [1, 5, 3, 8, 9, 7, 0, 100],
#                         [100, 100, 100, 100, 100, 100, 100, 0],
#                         ]
#     return data